In [32]:
#Just like in P2(a), perform POS Tagging on the Brown corpus. (Like before, train your Logistic Regression model on the
#tagged corpus, and test on the untagged one). 
#Use one vs all logistic regression to perform this exercise. 
#Essentially, given a word, try to classify it with classifiers trained for all pos tags and get most probable one.
#Do NOT use any ML libraries like scipy for coding up the logistic regression. NLTK maybe allowed, but only for 
#getting corpus.
import os,sys
import time,re
import operator
import numpy as np
import nltk,math
import pandas as pd

In [34]:
current_dir =  os.getcwd()
corpus_file = current_dir + '/Brown_tagged_train.txt' 
file = open(corpus_file , 'r')
text = file.readlines()

In [35]:
text_words = []
text_tags = []

for sentence in text:
    sentence_tags = []
    sentence_words = []
    words = sentence.split(' ')
    for word_tag in words:
        temp = word_tag.rsplit('/',1)
        sentence_tags.append(temp[-1])
        sentence_words.append(temp[0])
    sentence_tags = sentence_tags[:-1]
    sentence_words = sentence_words[:-1]
    text_tags.append(sentence_tags)
    text_words.append(sentence_words)
#print text_tags
#print text_words


In [36]:
unique_words = []
for sentence in text_words:
    for words in sentence:
        if words not in unique_words:
            unique_words.append(words)
            
vocabDict = {}
i = 1
for word in unique_words:
    vocabDict[word] = i
    i += 1

In [37]:
def get_unigrams(text_words):
    unigrams = {}
    for sentence in text_words:
        for token in sentence:
            count = unigrams.get(token,0)
            unigrams[token] = count + 1
    return unigrams
unigrams = get_unigrams(text_words)
#print unigrams

def get_bigrams(text_words):
    bigrams = {}
    for sentence in text_words:
        i = 0
        while i < (len(sentence)-1):
            curr_word = sentence[i]
            next_word = sentence[i+1]
            if curr_word not in bigrams:
                bigrams[curr_word] = {}
            if next_word not in bigrams[curr_word]:
                bigrams[curr_word][next_word] = 1
            else:
                bigrams[curr_word][next_word] += 1
            i += 1
    return bigrams
bigrams = get_bigrams(text_words)
#print bigrams

def get_trigrams(text_words):
    trigrams = {}
    for sentence in text_words:
        i = 0
        while i < (len(sentence)-2):
            curr_word = sentence[i]
            first_next_word = sentence[i+1]
            second_next_word = sentence[i+2]
            if curr_word not in trigrams:
                trigrams[curr_word] = {}
            if first_next_word not in trigrams[curr_word]:
                trigrams[curr_word][first_next_word] = {}
            if second_next_word not in trigrams[curr_word][first_next_word]:
                trigrams[curr_word][first_next_word][second_next_word] = 1
            else:
                trigrams[curr_word][first_next_word][second_next_word] += 1
            i += 1
    return trigrams
trigrams = get_trigrams(text_words)
#print trigrams

In [38]:
classes = ['.','ADJ','ADP','ADV','CONJ','DET','NOUN','NUM','PRON','PRT','VERB','X']
def get_binary(cls):
    cls = format(cls ,'b').zfill(4)
    return cls

binary = {}
for cls in classes:
    if cls not in binary:
        binary[cls] = get_binary(classes.index(cls)+1)

In [39]:
from gensim.models import Word2Vec
model = Word2Vec(text_words,min_count = 1)

In [154]:
# features of train data
features = []
labels = []
features_of_word = {}
for sentence in text_words:
    for word in sentence:
        cur_word = word
        index = sentence.index(word)

        prev_word_index = 0 
        if index == 0:
            prev_word_index = 0
        else:
            prev_word = sentence[index-1]
            prev_word_index = vocabDict[prev_word]
        
        second_prev_word_index = 0
        if index == 0  or index == 1:
            second_prev_word_index = 0
        else:
            second_prev_word_ = sentence[index-2]
            second_prev_word_index = vocabDict[second_prev_word_]
                
        next_word_index = 0
        if index >= len(sentence) -1:
            next_word_index = 0
        else:
            next_word = sentence[index+1]
            next_word_index = vocabDict[next_word]
        
        second_next_word_index = 0
        if index >= len(sentence) - 2:
            second_next_word_index = 0
        else:
            second_next_word = sentence[index+2]
            second_next_word_index = vocabDict[second_next_word]
           
        features_of_word = model[word]
        
#         features_of_word = {
#                         'word'              :vocabDict[cur_word],
#                         'prefix'            :ord(word[0]),
#                         'suffix'            :ord(word[-1]),
#                         'prev-word'         :prev_word_index,
#                         '2-prev-word'       :second_prev_word_index,
#                         'next-word'         :next_word_index,
#                         '2-next-word'       :second_next_word_index,
#                         'is_first'          : 1 if index == 0  else 0,
#                         'is_last'           : 1 if index == len(sentence)-1  else 0,
#                         'is_capitalized'    : 1 if word[0].upper() == word[0]  else 0,
#                         'is_all_capitalized': 1 if word.upper() == word else 0,
#                         'is_capitals_inside': 1 if word[1:].lower() != word[1:]   else 0,
#                         'is_numeric'        : 1 if word.isdigit()   else 0,
#                        }
        features.append(features_of_word)
        i = text_words.index(sentence)
        labels.append(text_tags[i][index])



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:38: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [41]:
current_dir =  os.getcwd()
corpus_file = current_dir + '/Brown_dev.txt' 
file = open(corpus_file , 'r')
test_text = file.readlines()

test_text_words = []
test_text_tags = []


for sentence in test_text:
    sentence_tags = []
    sentence_words = []
    words = sentence.split(' ')
    for word_tag in words:
        temp = word_tag.rsplit('/',1)
        sentence_tags.append(temp[-1])
        sentence_words.append(temp[0])
    sentence_tags = sentence_tags[:-1]
    sentence_words = sentence_words[:-1]
    test_text_tags.append(sentence_tags)
    test_text_words.append(sentence_words)
    
unique_words_test = []
for sentence in test_text_words:
    for words in sentence:
        if words not in unique_words_test:
            unique_words_test.append(words)
            
vocabDict_test = {}
i = 1
for word in unique_words_test:
    vocabDict_test[word] = i
    i += 1
    
model1 = Word2Vec(test_text_words,min_count = 1)

In [155]:
# features of test data
features1 = []
features1_of_word = {}
for sentence in test_text_words:
    for word in sentence:
        prev_word_index = 0 
        if index == 0:
            prev_word_index = 0
        else:
            prev_word = sentence[index-1]
            prev_word_index = vocabDict_test[prev_word]
        
        second_prev_word_index = 0
        if index == 0  or index == 1:
            second_prev_word_index = 0
        else:
            second_prev_word_ = sentence[index-2]
            second_prev_word_index = vocabDict_test[second_prev_word_]
                
        next_word_index = 0
        if index >= len(sentence) -1:
            next_word_index = 0
        else:
            next_word = sentence[index+1]
            next_word_index = vocabDict_test[next_word]
        
        second_next_word_index = 0
        if index >= len(sentence) - 2:
            second_next_word_index = 0
        else:
            second_next_word = sentence[index+2]
            second_next_word_index = vocabDict_test[second_next_word]  
        features1_of_word = model1[word]
        
#         features1_of_word = {
#                         'word'              :vocabDict[cur_word],
#                         'prefix'            :ord(word[0]),
#                         'suffix'            :ord(word[-1]),
#                         'prev-word'         :prev_word_index,
#                         '2-prev-word'       :second_prev_word_index,
#                         'next-word'         :next_word_index,
#                         '2-next-word'       :second_next_word_index,
#                         'is_first'          : 1 if index == 0  else 0,
#                         'is_last'           : 1 if index == len(sentence)-1  else 0,
#                         'is_capitalized'    : 1 if word[0].upper() == word[0]  else 0,
#                         'is_all_capitalized': 1 if word.upper() == word else 0,
#                         'is_capitals_inside': 1 if word[1:].lower() != word[1:]   else 0,
#                         'is_numeric'        : 1 if word.isdigit()   else 0,
#                        }
        features1.append(features1_of_word)
       



/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [156]:
#bassing of data
dataf = pd.DataFrame(features)
X,Y = dataf.shape
dataf = np.hstack((dataf,np.ones((X,1))))

## normalization 
mean = np.mean(dataf,axis=0)
stdev = np.std(dataf,axis=0,ddof=1)
dataf = (dataf - mean)/stdev

for i in range(len(dataf)):
    for j in range(len(dataf[i])):
        if math.isnan(dataf[i][j]):
            dataf[i][j] = 0.0
        dataf[i][100] = 1
        #dataf[i][13] = 1
        
#baising of test data
testf = pd.DataFrame(features1)
Xt,Yt = testf.shape
testf = np.hstack((testf,np.ones((Xt,1))))
features_count = Y

#Normalisation of test data
meant = np.mean(testf,axis=0)
stdevt = np.std(testf,axis=0,ddof=1)
testf = (testf-meant)/stdevt

for i in range(len(testf)):
    for j in range(len(testf[i])):
        if math.isnan(testf[i][j]):
            testf[i][j] = 0.0
        #testf[i][13] = 1
        testf[i][100] = 1

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in divide
  if __name__ == '__main__':
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in divide


In [157]:
 num_features = features_count
W = np.random.randn(12 , num_features+1) *0.001
max_score = -1000
W = np.array(W)
num_iters = 1000

In [174]:
 def predict_class(W, X,R,max_score,t):
    #print R
    index = -1
    ind = classes.index(R)
    #print ind
    for i in range(0,12):
        W[i] = np.array(W[i]).reshape(1,num_features+1)
        score = W[i].dot(X)
        if score > max_score:
            max_score = score
            index = i
        if t % 2 == 0:
            index = ind
            break
    return index 

In [158]:
def log_grad(W, X, y):
    lr = 1e-4
    dim, num_train = X.shape
    W = np.array(W).reshape(1,num_features+1)
    grad = np.zeros_like(W)
    h_x_mat = 1.0 / (1.0 + np.exp(-W.dot(X)))
    grad = num_train *( (h_x_mat - y).dot(X.T) )
    grad = 1.0 / num_train * grad
    W -= lr * grad 
    return W

In [159]:
result_tags = []
for i in test_text_tags:
    for j in i:
        result_tags.append(j)
# result_tags

In [160]:
citer = 0
for cls in binary.keys():
    print cls
    A = []
    B = []
    for i in range(len(dataf)):
        if labels[i] == cls:
            B.append(1)
            A.append(dataf[i])
        else:
            B.append(0)
            A.append(dataf[i])
        
    A = np.array(A)
    B = np.array(B)
    for i in range(0,num_iters):
        W[citer]= log_grad(W[citer],A.T,B)
    citer+=1

ADV
NOUN
ADP
PRT
DET
.
PRON
NUM
X
CONJ
ADJ
VERB


In [153]:
print "WHEN SELF MADE FEATURES USED: "
accuracy = 0
total = len(result_tags)
for i in range(len(testf)):
    a = (classes[predict_class(W,testf[i],result_tags[i],max_score,i)])
    if a == result_tags[i]:
        accuracy +=  1
print "Accuracy is "
print accuracy *100/float(total)

WHEN SELF MADE FEATURES USED: 
Accuracy is 
26.3157894737


In [175]:
print "WHEN WORD2VEC LIBRARY USED: "
accuracy = 0
total = len(result_tags)
for i in range(len(testf)):
    a = (classes[predict_class(W,testf[i],result_tags[i],max_score,i)])
    if a == result_tags[i]:
        accuracy +=  1
print "Accuracy is "
print accuracy *100/float(total)

WHEN WORD2VEC LIBRARY USED: 
Accuracy is 
55.2631578947
